In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.abspath('../src'))

from data_loader import DiabetesLoader
from adversarial_validation import Validator
from preprocessing import get_preprocessor

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

loader = DiabetesLoader('../data/raw/train.csv')
loader.load_external('../data/external/diabetes_dataset.csv')

pipeline = get_preprocessor()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loading data from /app/data/raw/train.csv
Data Loaded. Shape: (700000, 25)
loading external data from ../data/external/diabetes_dataset.csv


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import make_pipeline
from config import XGB_PARAMS, LGBM_PARAMS, CAT_PARAMS,TO_INTERACT_COLS, OPTIMAL_COLS, TARGET
from preprocessing import add_important_interaction

xgb = XGBClassifier(**XGB_PARAMS)
lgbm = LGBMClassifier(**LGBM_PARAMS)
cat = CatBoostClassifier(**CAT_PARAMS)



ensemble = VotingClassifier(
    estimators=[
        ('xgb',xgb),
        ('lgbm',lgbm),
        ('cat',cat)
    ],
    voting='soft',
    weights=[1,1,1],
    n_jobs=1
)



In [ ]:
X,y = loader.get_full()

X_processed =  pipeline.fit_transform(X)

print(f"full data x: {X_processed.shape}, y: {y.shape}")

validator = Validator(X_processed, 'is_external')
mask = validator.get_mask()

X_processed = X_processed[mask]
y = y[mask]

full data x: (770739, 34), y: (770739,)


In [ ]:
ensemble.fit(X_processed, y)

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='auc',
                                            feature_types=None,
                                            feature_weights=None,
                                            gamma=0.2829267954406464,
                                            grow_policy=None,
                                            importance_type=None,
                                            inte...
                                            multi_strategy=None,
                                            n_estimators=546, n_jobs=-1,
                                            num_parallel_tree=None, ...)),
                             ('lgbm',
                              LGBMClassifier(learning_rate=0.05186210130983924,
                                             max_depth=10, min_data_in_leaf=650,
                                             n_estimators=241, n_jobs=-1,
                                             num_leaves=925, random_state=42,
                                             scale_pos_weight=0.6049,
                                             verbose=-1)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x74f6b4449070>)],
                 n_jobs=1, voting='soft', weights=[1, 1, 1])

In [ ]:
from data_loader import TestLoader

test_loader = TestLoader('../data/raw/test.csv')
X_test, id = test_loader.get_data()
X_test['is_external'] = 0
X_test_processed =  pipeline.transform(X_test)

print(f"full test data x: {X_test_processed.shape}")

Loading data from /app/data/raw/test.csv
Test X: (300000, 25)
full test data x: (300000, 34)


In [ ]:
import pandas as pd

y_submit = ensemble.predict_proba(X_test_processed)[:, 1]
df_submit = pd.DataFrame(y_submit,columns=[TARGET], index=id)

display(df_submit.head())

df_submit.to_csv('../outputs/ensemble_external_no_fe.csv')

,diagnosed_diabetes
id,
700000,0.379330
700001,0.549680
700002,0.673436
700003,0.316995
700004,0.860742


In [28]:
print((df_submit[TARGET] > 0.95).sum())

741
